## Viewshed and solar energy potential analysis
Resources:

* [
GRASS GIS overview and manual](https://grass.osgeo.org/grass76/manuals/index.html)
* [ GRASSbook](http://www.grassbook.org/)


In [ ]:
# Import Python standard library and IPython packages we need.
import os
import subprocess
import sys

# Ask GRASS GIS where its Python packages are.
gisbase = subprocess.check_output(["grass", "--config", "path"], text=True).strip()
os.environ["GISBASE"] = gisbase
sys.path.append(os.path.join(gisbase, "etc", "python"))

# Import the GRASS GIS packages we need.
import grass.script as gs
import grass.jupyter as gj

# Start GRASS Session
gj.init("../../data/grassdata", "nc_basic_spm_grass7", "user1")

# Set computational region to elevation raster
gs.run_command('g.region', raster='elevation@PERMANENT', flags='pg')

### Viewshed analysis
Compute viewshed from a new 32 story tower located in downtown and from the former RedHat headquaters.

In [ ]:
gs.parse_command('g.region', raster="elevation", flags='apg')
gs.write_command('v.in.ascii', input='-', stdin='%s|%s' % (642212, 224767), output='viewpoints')
gs.run_command('r.viewshed', input="elevation", output="tower_los", coordinates="642212,224767", observer_elevation="165", max_distance="10000")

Display result on shaded relief:

In [ ]:
vs_map = gj.InteractiveMap()

vs_map.add_raster("tower_los", opacity=0.7)
vs_map.add_vector("viewpoints")
vs_map.add_layer_control()

vs_map.show()

### Solar radiation analysis

Set the region and add the planned building to the DEM, we will use this new DEM for the analyses.
Remove all layers and zoom to the region.

Prepare input maps (slope and aspect):

In [ ]:
gs.run_command('r.slope.aspect', elevation="elevation", aspect="aspect", slope="slope")

#### Incidence angles and cast shadows

Compute the sun position on Dec. 22 at 4:00pm, EST (no map output expected):

In [ ]:
gs.run_command('r.sunmask', elevation="elevation", year="2001", month="12", day="22", hour="16", minute="00", sec="0", timezone="-5", flags='s')

Calculate incidence angles including cast shadows.

In [ ]:
gs.run_command('r.sun', elevation="elevation", aspect="aspect", slope="slope", incidout="incident", day="356", time="16")
gs.parse_command('r.info', map="incident", flags='g')
gs.run_command('r.colors', map="incident", co="bcyr", flags='e')

solar_map = gj.GrassRenderer()
solar_map.d_rast(map="incident")
solar_map.d_legend(raster="incident", at="25,50,1,3")
solar_map.show()

Extract the cast shadow area for 4:00pm.

In [ ]:
gs.mapcalc("shadow = if(isnull(incident), 1, null())")
gs.run_command('r.colors', map="shadow", color="grey")
gs.run_command('r.colors', map="elevation", color="elevation")

shadow_map = gj.GrassRenderer()
shadow_map.d_rast(map="elevation")
shadow_map.d_rast(map="shadow")
shadow_map.show()

#### Solar radiation
Compute global (beam+diffuse+refl) radiation for entire day during summer and winter solstice.
Display the radiation maps.

In [ ]:
gs.run_command('r.sun', elevation="elevation", aspect="aspect", slope="slope", day="356", glob_rad="winter", insol_time="its356")
gs.run_command('r.colors', map="winter", co="gyr", flags='e')

gs.run_command('r.sun', elevation="elevation", aspect="aspect", slope="slope", day="172", glob_rad="summer", insol_time="its172")
gs.run_command('r.colors', map="summer", co="gyr", flags='e')

In [ ]:
rad_map = gj.InteractiveMap()

rad_map.add_raster("winter")
rad_map.add_raster("summer")
rad_map.add_layer_control()

rad_map.show()

In [ ]:
winter_map=gj.GrassRenderer()
winter_map.d_rast(map="winter")
winter_map.d_legend(raster="winter", at="25,50,1,3")
winter_map.show()

In [ ]:
summer_map = gj.GrassRenderer()
summer_map.d_rast(map="summer")
summer_map.d_legend(raster="summer", at="25,50,1,3", range="8800,8867")
summer_map.show()